In [2]:
'''
packages
'''
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import random
import pickle

Using TensorFlow backend.


In [4]:
#from keras_metrics import KerasMetrics

In [3]:
import coding_function as cf
import decoding_function as df
import csv
from Files import nodes

In [4]:
with open('Files/training_data_set.csv','r') as csvfile:
    readcsv = csv.reader(csvfile, delimiter = ',')
    phoneticinf = []
    phoneticI = []
    for row in readcsv:
        phoneticinf.append(row[1])
        phoneticI.append(row[3])

In [5]:
cf.dataTest(phoneticinf,phoneticI) # careful with duplicates on your dataset

'done'

In [12]:
# for key in dictioinf.keys(): 
#      if key in phoneticinf:
#         phoneticinf.remove(key)
# phoneticinf

In [6]:
dictioinf = {}
dictioI = {}

for item in phoneticinf:
    dictioinf[item] = cf.coding(item)

for item in phoneticI:
    dictioI[item] = cf.coding(item)

In [7]:
len(phoneticinf),len(phoneticI),len(dictioinf),len(dictioI)

(410, 410, 410, 410)

In [8]:
X = dictioinf.values() #input column
print (len(X))
Y = dictioI.values() #output column

410


In [9]:
X
np.array(X)
print(len(X),len(Y))

410 410


In [10]:
X = np.array(list(X))
Y = np.array(list(Y))

In [18]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 1, 0]])

In [11]:
'''
2. Define model
'''

model = Sequential()
model.add(Dense(460, input_shape=(460,), activation='sigmoid'))


In [20]:
'''
3. Evaluate model
'''



'\n3. Evaluate model\n'

In [12]:
'''
4. Compile model
'''

model.compile(loss='mean_squared_error', optimizer='adadelta')

In [14]:
'''
4. Fit model
'''
model.fit(X,Y, epochs=600, batch_size=20)

Epoch 1/500
410/410 [==============================] - 0s - loss: 0.0569     
Epoch 2/500
410/410 [==============================] - 0s - loss: 0.0568     
Epoch 3/500
410/410 [==============================] - 0s - loss: 0.0567     
Epoch 4/500
410/410 [==============================] - 0s - loss: 0.0565     

In [15]:
gostar = "#gosta#"
Gostar = cf.coding(gostar)
tomar = "#toma#"
Tomar = cf.coding(tomar)
mexer = "mexe"
Mexer = cf.coding(mexer)
Test_list = [Gostar,Tomar,Mexer]


In [16]:
Test_array = np.array(Test_list)
Test_array.shape

(3, 460)

In [17]:
prediction = model.predict(Test_array)
#print(prediction)

In [28]:
type(prediction)

numpy.ndarray

In [18]:
print(list(prediction[2]))

[0.12002007, 0.11238167, 0.33990109, 0.22778754, 0.41597572, 0.29275689, 0.23574434, 0.29338595, 0.33959806, 0.31820518, 0.29026893, 0.19961357, 0.95453292, 0.8914845, 0.69609827, 0.90034866, 0.42574048, 0.62764734, 0.95268333, 0.37400866, 0.22219548, 0.29516026, 0.3922089, 0.41435438, 0.31228411, 0.44307247, 0.1816078, 0.27059796, 0.23446621, 0.33421689, 0.096342042, 0.17686592, 0.59748262, 0.57597566, 0.45324889, 0.59274864, 0.29342195, 0.36368254, 0.59858054, 0.30069178, 0.23310819, 0.11642531, 0.42641568, 0.57095432, 0.20496838, 0.58466071, 0.19809705, 0.24490419, 0.47729051, 0.2928853, 0.40725672, 0.37744141, 0.36228412, 0.5281269, 0.29914206, 0.53418905, 0.27567425, 0.33540231, 0.34177539, 0.41178656, 0.26401374, 0.46472594, 0.21077295, 0.40581116, 0.34226939, 0.35267755, 0.3683663, 0.17158258, 0.44016907, 0.30116126, 0.21215412, 0.41050261, 0.22771285, 0.22954339, 0.35706919, 0.32469365, 0.20159926, 0.29530901, 0.45310968, 0.18864761, 0.74321562, 0.93804616, 0.18410006, 0.682961

In [493]:
thresholded = np.where(prediction >= 0.600, 1, 0)

In [1]:
prediction

NameError: name 'prediction' is not defined

In [494]:
lists = thresholded.tolist()

In [507]:
algo=lists[1]
algo.count(1)

51

In [508]:
algo

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [509]:
wickelfeatures_list = nodes.nds

In [510]:
resposta = cf.vector2string(algo,wickelfeatures_list)

In [511]:
df.decoding(resposta)

'#to'

In [457]:
df.decoding(cf.vector2string(cf.coding("#gOstu#"),wickelfeatures_list))

'#gOstu#'

In [499]:
cf.coding("#gosta#")

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
